<a href="https://colab.research.google.com/github/Felipecl22/Google_Capstone/blob/main/Gradient_Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Gradient Boosting**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Montando el drive para acceder a los archivos, ya que tengo los csv en mi Drive, esto no es necesario si ustedes tienen el archivo en su escritorio.

Mounted at /content/drive


In [ ]:
import pandas as pd

# Ruta del archivo CSV en Google Drive.
potencial_csv = '/content/drive/My Drive/Colab Notebooks/Módulo 5/potencial.xlsx'

# Importando la data y transformandola a objeto Dataframe.
# En este poner el path correspondiente del archivo como parametro de pd.read_csv.
df_potencial = pd.read_excel(potencial_csv)

# Imprimir la información del DataFrame consolidado
print(df_potencial)

        ID  Age  Income  ZIP Code  Family  CCAvg  Education  Mortgage  \
0        1   25      49     91107       4    1.6          1         0   
1        2   45      34     90089       3    1.5          1         0   
2        3   39      11     94720       1    1.0          1         0   
3        4   35     100     94112       1    2.7          2         0   
4        5   35      45     91330       4    1.0          2         0   
...    ...  ...     ...       ...     ...    ...        ...       ...   
4995  4996   29      40     92697       1    1.9          3         0   
4996  4997   30      15     92037       4    0.4          1        85   
4997  4998   63      24     93023       2    0.3          3         0   
4998  4999   65      49     90034       3    0.5          2         0   
4999  5000   28      83     92612       3    0.8          1         0   

      Personal Loan  
0                 0  
1                 0  
2                 0  
3                 0  
4            

Me parece que utilizar las columnas Income, CCavg y Mortgage para predecir si un cliente tomará un Personal loan.

"CCAvg" representa el gasto promedio en tarjeta de crédito.

"Mortgage" representa el saldo pendiente del préstamo hipotecario en el conjunto de datos dado.

In [ ]:
# Verificar por variables categóricas

categoricas = []

for var in df_potencial:
    if df_potencial[var].dtype == 'O':
        categoricas.append(var)

print(f'Hay {len(categoricas)} variables categóricas en la dataframe')
print(f'Estas son: {categoricas}')

Hay 0 variables categóricas en la dataframe
Estas son: []


In [ ]:
# Verificar por variables categóricas

numericas = []

for var in df_potencial:
    if df_potencial[var].dtype != 'O':
        numericas.append(var)

print(f'Hay {len(numericas)} variables categóricas en la dataframe')
print(f'Estas son: {numericas}')

Hay 9 variables categóricas en la dataframe
Estas son: ['ID', 'Age', 'Income', 'ZIP Code', 'Family', 'CCAvg', 'Education', 'Mortgage', 'Personal Loan']


In [ ]:
# Verificamos por valores únicos
df_potencial['Personal Loan'].unique()

array([0, 1])

Con lo anterior, estamos listos para crear un Gradient Boosting regresivo.

In [ ]:
# Creando variables predictorias y variable objetivo.
X = df_potencial[['Income', 'CCAvg', 'Mortgage']]
y = df_potencial['Personal Loan']

In [ ]:
X.isnull().sum()

Income      0
CCAvg       0
Mortgage    0
dtype: int64

In [ ]:
y.isnull().sum()

0

Normalizar datos

In [ ]:
from sklearn.preprocessing import StandardScaler

# Guarda los nombres de las columnas antes de la transformación
original_columns = X.columns.tolist()

# Inicializa y ajusta el scaler a tus datos
scaler = StandardScaler()
scaler.fit(X)

# Transforma tus datos utilizando el scaler ajustado
X_scaled = scaler.transform(X)

# Convierte la matriz transformada en un DataFrame de pandas manteniendo los nombres de las columnas originales
X_df = pd.DataFrame(X_scaled, columns=original_columns)

In [ ]:
X_df

,Income,CCAvg,Mortgage
0,-0.538229,-0.193371,-0.555524
1,-0.864109,-0.250595,-0.555524
2,-1.363793,-0.536720,-0.555524
3,0.569765,0.436103,-0.555524
4,-0.625130,-0.536720,-0.555524
...,...,...,...
4995,-0.733757,-0.021696,-0.555524
4996,-1.276892,-0.880069,0.280238
4997,-1.081363,-0.937294,-0.555524
4998,-0.538229,-0.822844,-0.555524


In [ ]:
y.unique()

array([0, 1])

In [ ]:
from sklearn.model_selection import train_test_split
# División de los datos en conjunto de entrenamiento y prueba en una proporción de 5:1
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

# Inicializar el modelo de Gradient Boosting para regresión
gb_regressor = GradientBoostingRegressor()

# Entrenar el modelo utilizando el conjunto de entrenamiento
gb_regressor.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predictions = gb_regressor.predict(X_test)

# Calcular el error cuadrático medio (MSE) en el conjunto de prueba
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

# Realizar validación cruzada para obtener una evaluación más robusta del modelo
cv_scores = cross_val_score(gb_regressor, X, y, cv=5, scoring='neg_mean_squared_error')
cv_mse = -cv_scores.mean()  # Negamos el MSE para obtener el valor real
print("Cross-Validated Mean Squared Error:", cv_mse)

# También puedes evaluar otras métricas de rendimiento según tus necesidades

# Si deseas obtener una interpretación del modelo, puedes examinar la importancia de las características
feature_importances = gb_regressor.feature_importances_
print("Feature Importances:", feature_importances)

Mean Squared Error: 0.05134892195707834
Cross-Validated Mean Squared Error: 0.0520731598529395
Feature Importances: [0.64788417 0.28853495 0.06358089]


El error cuadrático medio (MSE) obtenido en el conjunto de prueba es aproximadamente 0.0514. Esto indica que, en promedio, las predicciones del modelo están a una distancia de 0.0514 unidades al cuadrado de las etiquetas reales del conjunto de prueba. En general, cuanto más bajo sea el valor del MSE, mejor será el rendimiento del modelo.

Las importancias de las características. Estas son las ponderaciones que el modelo asigna a cada característica para hacer predicciones. En este caso, se tienen tres características y las importancias son aproximadamente 0.6474, 0.2886 y 0.0640 para la primera, segunda y tercera característica respectivamente.

Estas importancias indican qué características son más influyentes en las predicciones del modelo. Una importancia más alta significa que la característica tiene un mayor impacto en las predicciones del modelo. Esto puede ser útil para comprender qué características son más relevantes para el problema en cuestión y para realizar selección de características si es necesario.

El "Cross-Validated Mean Squared Error" (Error Cuadrático Medio Validado Cruzadamente) es una medida del rendimiento del modelo de regresión después de aplicar la validación cruzada. Es el promedio del error cuadrático medio (MSE) obtenido en todas las particiones del conjunto de datos durante la validación cruzada.

En términos simples, representa cuán cerca están las predicciones del modelo de los valores reales en promedio, después de tener en cuenta la variabilidad en los datos que puede surgir de diferentes divisiones del conjunto de datos en conjuntos de entrenamiento y prueba.

En tu caso, un "Cross-Validated Mean Squared Error" de 0.052 indica que, en promedio, el modelo predice el cuadrado de la diferencia entre las predicciones y los valores reales de la variable objetivo (en este caso, probablemente el préstamo personal). Un valor de MSE más bajo indica un mejor rendimiento del modelo, ya que implica que las predicciones están más cerca de los valores reales. Por lo tanto, un MSE de 0.052 es bastante bajo y sugiere que el modelo tiene un buen rendimiento en la predicción de la variable objetivo.

In [ ]:
# Suponiendo que 'X_test' es tu conjunto de características de prueba

# Realiza predicciones en el conjunto de prueba
predictions = gb_regressor.predict(X_test)

# Muestra las predicciones
print("Predicciones:", predictions)

Predicciones: [ 7.94257414e-04  3.90938455e-01  4.62305308e-01  7.94257414e-04
  7.94257414e-04 -1.27925688e-03  3.12219832e-01  1.57673762e-01
  7.94257414e-04  3.15980952e-01  7.94257414e-04  7.94257414e-04
  4.79519218e-01  9.54692598e-04  4.34619005e-02  1.03728833e+00
  5.19794811e-02  6.45520470e-01  7.94257414e-04  9.07033849e-01
  7.94257414e-04  3.64793550e-01  3.08455161e-01  4.59173583e-02
  7.94257414e-04  7.94257414e-04  7.94257414e-04  3.34408333e-01
  8.75460979e-02  3.37735671e-01  4.06784563e-01  7.94257414e-04
  9.05731331e-01  1.53113737e-01  7.94257414e-04  7.94257414e-04
  7.94257414e-04  7.94257414e-04  7.94257414e-04  7.94257414e-04
  7.94257414e-04 -1.27925688e-03  3.05763944e-01  3.72066681e-01
  7.94257414e-04  9.54692598e-04  5.21160229e-03 -5.02395209e-03
  1.79048155e-01  7.94257414e-04  2.45782027e-03  3.84730403e-01
  4.65789279e-01  4.59173583e-02  1.04598753e-02  7.94257414e-04
  7.94257414e-04  7.94257414e-04  7.94257414e-04  4.03936126e-01
  2.8814385

La predicción para la primera fila es aproximadamente 0.0008. Esto significa que el modelo estima que hay una probabilidad muy baja, alrededor del 0.08%, de que el cliente correspondiente en la primera fila acepte un préstamo personal.

In [ ]:

# Crear un nuevo cliente con características específicas en un DataFrame
nuevo_cliente = pd.DataFrame({'Income': [-0.7], 'CCAvg': [0.2], 'Mortgage': [-0.3]})

# Realizar la predicción utilizando el modelo entrenado
prediccion = gb_regressor.predict(nuevo_cliente)

# Imprimir la predicción
print("Personal Loan Probability:", prediccion)

Personal Loan Probability: [0.00095469]
